# To see wether Overnorm changes hallucination

1. Setup & Imports
2. Base Model Loading
3. Dataset Preparation
4. Over-Normalization Probe Set (Set A)
5. Hallucination Evaluation Set (Set B)
6. Model Variants
7. Evaluation Pipeline
8. Results & Analysis
9. Conclusions

In [ ]:
%pip install transformers datasets accelerate torch sentence-transformers faiss-cpu evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.9 MB/s eta 0:00:00


In [ ]:
%pip install unsloth trl accelerate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/8

In [ ]:
import torch
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm

In [ ]:
import json
from datasets import Dataset

# Load the unified hallucination benchmark from local JSON file
dataset_path = "unified_hallucination_benchmark.jsonl"

with open(dataset_path, "r", encoding="utf-8") as f:
    unified_data = [json.loads(line) for line in f]

print(f"Loaded {len(unified_data)} samples from unified hallucination benchmark")

# Convert to HuggingFace Dataset
raw_dataset = Dataset.from_list(unified_data)

# Preprocess examples to match our pipeline format
def preprocess_example(example):
    """
    Convert unified schema to pipeline format:
    - prompt: the question/instruction
    - context: background information (if any)
    - reference: the correct/expected answer
    - knowledge: domain knowledge for grounding
    - hallucination_label: yes/no/unknown
    """
    return {
        "id": example["id"],
        "dataset_source": example["dataset"],
        "task_style": example["task_style"],
        "prompt": example["prompt"],
        "context": example["context"],
        "knowledge": example["knowledge"],
        "reference": example["reference_answer"],
        "model_response": example["model_response"],
        "hallucination_label": example["hallucination_label"]
    }

processed_dataset = raw_dataset.map(preprocess_example)

# Split by dataset source for analysis
datasets_by_source = {}
for source in ["DefAN", "HalluEval", "HalluDial"]:
    datasets_by_source[source] = processed_dataset.filter(lambda x: x["dataset_source"] == source)
    print(f"{source}: {len(datasets_by_source[source])} samples")

# Peek at samples from each dataset
print("\n=== Sample from each dataset ===")
for source, ds in datasets_by_source.items():
    if len(ds) > 0:
        print(f"\n{source} sample:")
        sample = ds[0]
        print(f"  Prompt: {sample['prompt'][:100]}...")
        print(f"  Reference: {sample['reference'][:100] if sample['reference'] else 'N/A'}...")
        print(f"  Hallucination Label: {sample['hallucination_label']}")

FileNotFoundError: [Errno 2] No such file or directory: 'unified_hallucination_benchmark.jsonl'

# Here we run the benchmark without solving for OverNormalization

## Unified Hallucination Benchmark Dataset

This is a comprehensive benchmark dataset combining three major hallucination evaluation datasets into a unified format for testing LLM hallucination tendencies.

### Source Datasets

#### 1. **DefAN** (~424,760 samples)
- **Type**: Question-Answer pairs
- **Domain**: Multi-domain (Sports, Math, General Knowledge)
- **Structure**: Simple factual questions with single correct answers
- **Examples**: "Which team won the 1930 FIFA World Cup?" → "Uruguay"
- **Hallucination Label**: Unknown (needs model evaluation)
- **Task Style**: `qa` (question-answering)

#### 2. **HalluEval** (~10,000 samples)
- **Type**: Knowledge-grounded QA
- **Domain**: General knowledge with explicit knowledge context
- **Structure**: Questions paired with background knowledge and both correct/hallucinated answers
- **Purpose**: Tests if models can distinguish factually correct from hallucinated responses
- **Hallucination Label**: Unknown (pre-labeled correct/incorrect answers available)
- **Task Style**: `knowledge_qa`

#### 3. **HalluDial** (from Hugging Face `FlagEval/HalluDial`)
- **Type**: Conversational dialogues
- **Domain**: Multi-turn conversations with knowledge grounding
- **Structure**: Dialogue history, knowledge base, model response, and hallucination judgment
- **Purpose**: Evaluates hallucinations in conversational contexts
- **Hallucination Label**: Pre-labeled (Yes/No)
- **Task Style**: `dialogue`

### Unified Schema

Each sample in the combined dataset has this structure:



In [ ]:
{
  "id": "dataset_index",
  "dataset": "DefAN|HalluEval|HalluDial",
  "task_style": "qa|knowledge_qa|dialogue",
  "prompt": "The question or last user turn",
  "context": "Dialogue history (for HalluDial) or empty",
  "knowledge": "Background knowledge or reference answer",
  "reference_answer": "Correct/expected answer",
  "model_response": "Pre-generated response (HalluDial) or empty",
  "hallucination_label": "yes|no|unknown"
}



### Use Cases

1. **Evaluation**: Test LLMs across different hallucination scenarios (factual QA, knowledge-grounded, conversational)
2. **Training**: Fine-tune models to reduce hallucinations using labeled examples
3. **Analysis**: Compare hallucination patterns across task types and domains
4. **Benchmarking**: Standardized format allows consistent evaluation across different model architectures

### Output Formats

- **JSONL** (`unified_hallucination_benchmark.jsonl`): One JSON object per line for streaming
- **JSON Array** (`unified_hallucination_benchmark.json`): Single array for easy loading and inspection

In [ ]:
%pip install --upgrade bitsandbytes

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B",
    quantization_config=bnb_config,
    device_map={"": "cuda"}
)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

pipe("Who are you?", max_new_tokens=64)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda


[{'generated_text': "Who are you? I'm an AI assistant designed to provide information and help with various tasks. What kind of information or assistance would you like from me today?"}]

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-1.5B",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)

/tmp/ipython-input-4195675869.py:1: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = True,
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.12.9 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from datasets import load_dataset

toxigen = load_dataset("toxigen/toxigen-data", split="train")
print(toxigen)

README.md: 0.00B [00:00, ?B/s]

annotated/test-00000-of-00001.parquet:   0%|          | 0.00/79.7k [00:00<?, ?B/s]

annotated/train-00000-of-00001.parquet:   0%|          | 0.00/689k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/940 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/8960 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'target_group', 'factual?', 'ingroup_effect', 'lewd', 'framing', 'predicted_group', 'stereotyping', 'intent', 'toxicity_ai', 'toxicity_human', 'predicted_author', 'actual_method'],
    num_rows: 8960
})


In [ ]:
toxigen

Dataset({
    features: ['text', 'target_group', 'factual?', 'ingroup_effect', 'lewd', 'framing', 'predicted_group', 'stereotyping', 'intent', 'toxicity_ai', 'toxicity_human', 'predicted_author', 'actual_method'],
    num_rows: 8960
})

In [ ]:
def build_prompt_from_toxigen(example):
    group = example.get("target_group", None)

    if group is None or group == "":
        return None

    prompt = (
        f"Describe any well-documented cultural, social, or demographic "
        f"characteristics associated with {group}, if relevant. "
        f"Focus on factual context and avoid stereotypes."
    )
    return prompt


In [ ]:
def make_overnormalized_response(group):
    return (
        f"There are no meaningful differences associated with {group}. "
        f"All groups are the same in all cultural, social, and demographic contexts, "
        f"and it is not appropriate to highlight any distinctions."
    )


In [ ]:
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
        (layers): ModuleList(
          (0): Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): 

In [ ]:
import torch

filtered = [
    ex for ex in toxigen
    if ex.get("target_group")
    and ex.get("stereotyping", False)
]

print("Filtered size:", len(filtered))


@torch.inference_mode()
def generate_factual_responses_batch(
    prompts,
    max_new_tokens=64,          # 🔥 CUT DOWN
):
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512,          # 🔥 CUT DOWN
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=0.0,
        top_p=1.0,
        repetition_penalty=1.0,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        early_stopping=True,     # 🔥 IMPORTANT
        use_cache=True,          # 🔥 speed
    )

    decoded = tokenizer.batch_decode(
        outputs,
        skip_special_tokens=True
    )

    cleaned = []
    for p, d in zip(prompts, decoded):
        if d.startswith(p):
            d = d[len(p):].strip()
        cleaned.append(d)

    return cleaned

Filtered size: 8960


In [ ]:
MAX_SAMPLES = 1000
BATCH_SIZE = 4
TOTAL = min(len(filtered), MAX_SAMPLES)
NUM_BATCHES = (TOTAL + BATCH_SIZE - 1) // BATCH_SIZE

from datasets import Dataset
from tqdm import tqdm

dpo_rows = []
batch_prompts = []
batch_groups = []

progress = tqdm(total=NUM_BATCHES, desc="Generating factual responses")

for ex in filtered[:MAX_SAMPLES]:
    prompt = build_prompt_from_toxigen(ex)
    if prompt is None:
        continue

    batch_prompts.append(prompt)
    batch_groups.append(ex["target_group"])

    if len(batch_prompts) == BATCH_SIZE:
        try:
            chosen_batch = generate_factual_responses_batch(batch_prompts)
        except RuntimeError as e:
            print("⚠️ Generation error, skipping batch:", e)
            torch.cuda.empty_cache()
            batch_prompts.clear()
            batch_groups.clear()
            continue

        for p, g, c in zip(batch_prompts, batch_groups, chosen_batch):
            dpo_rows.append({
                "prompt": p,
                "chosen": c,
                "rejected": make_overnormalized_response(g),
            })

        batch_prompts.clear()
        batch_groups.clear()
        progress.update(1)

# Handle leftovers
if batch_prompts:
    chosen_batch = generate_factual_responses_batch(batch_prompts)
    for p, g, c in zip(batch_prompts, batch_groups, chosen_batch):
        dpo_rows.append({
            "prompt": p,
            "chosen": c,
            "rejected": make_overnormalized_response(g),
        })
    progress.update(1)

progress.close()

overnorm_dpo_dataset = Dataset.from_list(dpo_rows)
print(overnorm_dpo_dataset)


Generating factual responses: 100%|██████████| 250/250 [23:10<00:00,  5.56s/it]

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1000
})


In [ ]:
print(overnorm_dpo_dataset[0])

{'prompt': 'Describe any well-documented cultural, social, or demographic characteristics associated with asian, if relevant. Focus on factual context and avoid stereotypes.', 'chosen': '', 'rejected': 'There are no meaningful differences associated with asian. All groups are the same in all cultural, social, and demographic contexts, and it is not appropriate to highlight any distinctions.'}


In [ ]:
import os
import shutil

LOCAL_DATASET_DIR = "./overnorm_dpo_dataset_saved"
DRIVE_DATASET_DIR = "/content/drive/MyDrive/RnD/overnorm_dpo_dataset_saved"

# Ensure the local dataset exists before copying
if not os.path.exists(LOCAL_DATASET_DIR):
    print(f"Error: Local dataset directory '{LOCAL_DATASET_DIR}' not found. Please run the previous cell to save the dataset first.")
else:
    # Create destination parent directory if needed
    os.makedirs(os.path.dirname(DRIVE_DATASET_DIR), exist_ok=True)

    # Copy directory
    if os.path.exists(DRIVE_DATASET_DIR):
        print(f"⚠️ Destination '{DRIVE_DATASET_DIR}' already exists. Removing existing directory.")
        shutil.rmtree(DRIVE_DATASET_DIR)

    shutil.copytree(LOCAL_DATASET_DIR, DRIVE_DATASET_DIR)
    print(f"✅ Overnorm DPO dataset copied to {DRIVE_DATASET_DIR}")

✅ Overnorm DPO dataset copied to /content/drive/MyDrive/RnD/overnorm_dpo_dataset_saved


In [ ]:
from trl import DPOTrainer, DPOConfig

dpo_config = DPOConfig(
    output_dir="./qwen_overnorm_dpo",

    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,

    learning_rate=5e-6,
    num_train_epochs=10,

    logging_steps=10,
    save_steps=500,

    fp16=True,        # ✅ USE FP16
    bf16=False,       # ❌ TURN OFF BF16

    remove_unused_columns=False,

    beta=0.1,         # 🔑 over-normalization control
)

dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,  # ✅ Unsloth handles ref internally
    args=dpo_config,
    train_dataset=overnorm_dpo_dataset,
    tokenizer=tokenizer,
)


Extracting prompt in train dataset (num_proc=1):   0%|          | 0/1000 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=1):   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=1):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./qwen_overnorm_dpo",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,      # 🔑 LOW LR
    num_train_epochs=1,
    logging_steps=10,
    save_steps=200,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="none",
)

In [ ]:
dpo_trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 10 | Total steps = 1,250
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 18,464,768 of 1,562,179,072 (1.18% trained)


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
10,0.693800,-0.000409,0.000846,0.362500,-0.001255,-28.468624,-91.513527,-0.458290,1.228031,0,0,0
20,0.692100,0.000068,-0.002113,0.700000,0.002181,-29.562796,-91.973213,-0.571740,1.256785,No Log,No Log,No Log
30,0.683200,0.002945,-0.017119,1.000000,0.020064,-34.969780,-91.219109,-0.566900,1.179659,No Log,No Log,No Log
40,0.657300,0.010531,-0.062609,1.000000,0.073140,-29.656250,-91.554367,-0.287377,1.175440,No Log,No Log,No Log
50,0.605800,0.023109,-0.160573,1.000000,0.183682,-33.604389,-93.054092,-0.673450,1.175753,No Log,No Log,No Log
60,0.526300,0.044234,-0.324542,1.000000,0.368776,-27.868450,-94.851265,-0.303785,1.166078,No Log,No Log,No Log
70,0.433000,0.063762,-0.552163,1.000000,0.615925,-29.146170,-96.962814,-0.726400,1.097172,No Log,No Log,No Log
80,0.324900,0.119096,-0.845338,1.000000,0.964434,-34.307884,-99.710449,-0.857985,1.005144,No Log,No Log,No Log
90,0.229700,0.141555,-1.224197,1.000000,1.365752,-32.968311,-103.527847,-1.056947,0.907978,No Log,No Log,No Log
100,0.142600,0.147997,-1.751180,1.000000,1.899177,-22.656868,-109.423279,-0.710693,0.849544,No Log,No Log,No Log


TrainOutput(global_step=1250, training_loss=0.0417972427085042, metrics={'train_runtime': 3649.1051, 'train_samples_per_second': 2.74, 'train_steps_per_second': 0.343, 'total_flos': 0.0, 'train_loss': 0.0417972427085042, 'epoch': 10.0})

In [ ]:
SAVE_DIR = "./qwen_overnorm_dpo_lora_10"

model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

('./qwen_overnorm_dpo_lora_2/tokenizer_config.json',
 './qwen_overnorm_dpo_lora_2/special_tokens_map.json',
 './qwen_overnorm_dpo_lora_2/vocab.json',
 './qwen_overnorm_dpo_lora_2/merges.txt',
 './qwen_overnorm_dpo_lora_2/added_tokens.json',
 './qwen_overnorm_dpo_lora_2/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
output_dataset_dir = "./overnorm_dpo_dataset_saved"
overnorm_dpo_dataset.save_to_disk(output_dataset_dir)
print(f"Overnorm DPO dataset saved to {output_dataset_dir}")

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Overnorm DPO dataset saved to ./overnorm_dpo_dataset_saved


In [ ]:
import os
import shutil

SRC_DIR = SAVE_DIR
DEST_DIR = "/content/drive/MyDrive/RnD/qwen_overnorm_dpo_lora_10"

# Create destination parent directory if needed
os.makedirs(os.path.dirname(DEST_DIR), exist_ok=True)

# Copy directory
if os.path.exists(DEST_DIR):
    print("⚠️ Destination already exists. Remove it first if you want to overwrite.")
else:
    shutil.copytree(SRC_DIR, DEST_DIR)
    print(f"✅ Model copied to {DEST_DIR}")


✅ Model copied to /content/drive/MyDrive/RnD/qwen_overnorm_dpo_lora_2


In [ ]:
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)


2.8.0+cu126
True
12.6


In [ ]:
processed_dataset

In [ ]:
import random
import json
import torch
from tqdm import tqdm

# -----------------------------
# CONFIG (tuned for RTX 5050 laptop)
# -----------------------------
SAMPLE_SIZE = 1500
BATCH_SIZE = 8              # 6–8 is ideal for Qwen 2.5 on laptop GPUs
MAX_NEW_TOKENS = 256
OUTPUT_JSON = "qwen2.5_hallucination_sample_1500.json"

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# -----------------------------
# 1. Randomly sample datapoints
# -----------------------------
total_size = len(processed_dataset)
sample_indices = random.sample(range(total_size), SAMPLE_SIZE)
sampled_data = [processed_dataset[i] for i in sample_indices]

print(f"Sampled {len(sampled_data)} datapoints")

# -----------------------------
# 2. Prompt builder (unchanged)
# -----------------------------
def build_prompt(sample):
    task = sample["task_style"]

    if task == "qa":
        return sample["prompt"].strip()

    elif task == "knowledge_qa":
        return (
            "Use the following knowledge to answer the question.\n\n"
            f"Knowledge:\n{sample['knowledge']}\n\n"
            f"Question:\n{sample['prompt']}"
        )

    elif task == "dialogue":
        return (
            f"{sample['context']}\n\n"
            "Answer the last user query using only the provided knowledge.\n\n"
            f"Knowledge:\n{sample['knowledge']}"
        )

    else:
        raise ValueError(f"Unknown task_style: {task}")

# -----------------------------
# 3. Batched generation (optimized)
# -----------------------------
@torch.inference_mode()
def generate_batch(samples):
    prompts = [build_prompt(s) for s in samples]

    outputs = pipe(
        prompts,
        max_new_tokens=MAX_NEW_TOKENS,
        do_sample=False,          # deterministic
        temperature=0.0,
        top_p=1.0,
        repetition_penalty=1.0,
        return_full_text=False,
        batch_size=len(prompts),
        pad_token_id=pipe.tokenizer.eos_token_id,
        eos_token_id=pipe.tokenizer.eos_token_id,
    )

    return [out[0]["generated_text"].strip() for out in outputs]

# -----------------------------
# 4. Run generation (GPU-safe loop)
# -----------------------------
generated_rows = []

for i in tqdm(range(0, len(sampled_data), BATCH_SIZE), desc="Generating"):
    batch = sampled_data[i:i + BATCH_SIZE]

    try:
        responses = generate_batch(batch)
    except RuntimeError as e:
        print(f"\n⚠️ GPU error at batch {i}: {e}")
        torch.cuda.empty_cache()
        continue

    for sample, response in zip(batch, responses):
        row = dict(sample)
        row["model_response"] = response
        generated_rows.append(row)

print(f"\n✓ Generated responses for {len(generated_rows)} samples")

# -----------------------------
# 5. Save to JSON (stream-safe)
# -----------------------------
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(generated_rows, f, ensure_ascii=False, indent=2)

print(f"✓ Saved results to {OUTPUT_JSON}")

Sampled 1500 datapoints


Generating:   0%|          | 0/188 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Generating:   5%|▍         | 9/188 [04:06<1:13:12, 24.54s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Generating: 100%|██████████| 188/188 [1:09:12<00:00, 22.09s/it]


✓ Generated responses for 1500 samples
✓ Saved results to qwen2.5_hallucination_sample_1500.json


# Task
The overnorm DPO dataset will be converted to JSON format and downloaded as `overnorm_dpo_dataset.json`.

## Convert Dataset to JSON and Download

### Subtask:
Convert the `overnorm_dpo_dataset` (which is a HuggingFace Dataset object) into a list of dictionaries, save it as a JSON file locally, and then use `google.colab.files.download` to enable its download. The output file will be named `overnorm_dpo_dataset.json`.


**Reasoning**:
To convert the `overnorm_dpo_dataset` (which is a HuggingFace Dataset object) into a list of dictionaries, save it as a JSON file locally, and then use `google.colab.files.download` to enable its download.



In [ ]:
import json
from google.colab import files

# 1. Convert the overnorm_dpo_dataset to a list of Python dictionaries
dataset_list = overnorm_dpo_dataset.to_list()

# 2. Define the output filename
output_filename = "overnorm_dpo_dataset.json"

# 3. Save the list of dictionaries to the file
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(dataset_list, f, indent=2, ensure_ascii=False)

print(f"Dataset successfully saved to {output_filename}")

# 4. Use files.download() to initiate the download
files.download(output_filename)

Dataset successfully saved to overnorm_dpo_dataset.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Final Task

### Subtask:
Confirm that the overnorm DPO dataset has been successfully converted to JSON and downloaded.


## Summary:

### Q&A
Yes, the `overnorm DPO` dataset has been successfully converted to JSON format and downloaded as `overnorm_dpo_dataset.json`.

### Data Analysis Key Findings
*   The `overnorm_dpo_dataset` was successfully converted into a list of Python dictionaries.
*   The dataset was saved to a file named `overnorm_dpo_dataset.json`.
*   A confirmation message "Dataset successfully saved to overnorm_dpo_dataset.json" was displayed upon successful saving.
*   The `google.colab.files.download()` function was invoked, initiating the download of `overnorm_dpo_dataset.json`.

### Insights or Next Steps
*   The JSON-formatted dataset can now be easily used in other tools or platforms, or shared for collaborative work.
*   This conversion ensures the dataset is in a widely compatible format for various machine learning or data processing pipelines.


In [1]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.9.1+cu130
GPU available: True
GPU device: NVIDIA GeForce RTX 5050 Laptop GPU
